In [544]:
import os
import re
import json
import numpy as np
from random import choice
from pymongo import MongoClient
from itertools import combinations
from sage.all import *

In [213]:
to_list = lambda x: json.loads(x.replace('{', '[').replace('}', ']'))
doc_to_mat = lambda x: np.stack(to_list(x))

In [278]:
def get_sr_ideal(D, dresverts, triang):
    sr = []
    for n in range(1, 5):
        for c in combinations(range(len(dresverts)), n):
            for t in triang:
                if set(c).issubset(set(t)):
                    break
            else:
                for s in sr:
                    if set(c).issuperset(set(s)):
                        break
                else:
                    sr.append(c)
    return [prod(D[i] for i in c) for c in sr]

In [541]:
uri = os.getenv('MONGO_URI')
client = MongoClient(uri)
mdb = client.ToricCY

In [595]:
h11=3
docs = list(
    mdb.INVOL.find({
        "H11": h11
    })
)
doc = choice(docs)
doc.update(
    mdb.TRIANG.find_one({
        "POLYID": doc['POLYID'],
        "GEOMN": doc['GEOMN'],
        "TRIANGN": doc['TRIANGN']
    })
)
doc.update(
    mdb.GEOM.find_one({
        "POLYID": doc['POLYID'],
        "GEOMN": doc['GEOMN']
    })
)
doc.update(
    mdb.POLY.find_one({
        "POLYID": doc['POLYID']
    })
)

In [596]:
dresverts, rescws, triang = map(to_mat, map(doc.__getitem__, ['DRESVERTS', 'RESCWS', 'TRIANG']))

In [597]:
fan = Fan([Cone(dresverts[x].tolist()) for x in triang])
A = ToricVariety(fan)
A.inject_variables()
z = A.gens()
R = A.cohomology_ring()
D = R0.gens()

Defining z0, z1, z2, z3, z4, z5, z6


In [598]:
doc['INVOL']

'{D1->D2,D2->D1,D3->D4,D4->D3,D6->D7,D7->D6}'

In [632]:
doc['OPLANES']

[{'OIDEAL': ['-x2*x3+x1*x4', 'x1^2*x6-x2^2*x7'], 'ODIM': 5},
 {'OIDEAL': ['-x2*x3+x1*x4', 'x1^2*x6+x2^2*x7'], 'ODIM': 5}]

In [688]:
a, b = (DD[2-1]+DD[3-1]), (2*DD[1-1]+DD[6-1])

In [692]:
prod(DD[i] for i in triang[0])

-1/6*J2bar^4

In [703]:
norms = [prod(DD[i] for i in x)*abs(Matrix([dresverts[i] for i in x]).det()) for x in triang]
print(all(x==norms[0] for x in norms))
norm = norms[0]

True


In [697]:
[abs(Matrix([dresverts[i] for i in x]).det()) for x in triang]

[2, 2, 4, 2, 2, 4, 2, 2, 4, 2, 2, 4]

In [704]:
# Euler char?
(a+b)*a*b*sum(DD)/norm

6

In [647]:
sum(DD)

8*J1bar + 4*J2bar

In [674]:
b = A.linear_equivalence_ideal()

In [678]:
b = A.fan()

In [681]:
b = A.toric_divisor_group()

In [663]:
A.cohomology_ring().gens()

([z3], [z6], [-2*z3 + 1/2*z5], [z3], [1/2*z5 - 2*z6], [z5], [z6])

In [664]:
A.integrate(sum(A.cohomology_ring().gens())**4)

256

In [665]:
A.integrate(A.volume_class())

1

In [668]:
A.fan().generating_cone(0)

4-d cone of Rational polyhedral fan in 4-d lattice N

In [666]:
A.dimension_relative()

4

In [651]:
prod(1+x for x in DD)#.reduce(chow_ideal)

-5/3*J2bar^4 - 6*J0bar*J2bar^2 + 42*J1bar*J2bar^2 + 8*J2bar^3 + 4*J0bar*J1bar + 2*J0bar*J2bar + 22*J1bar*J2bar + 5*J2bar^2 + 8*J1bar + 4*J2bar + 1

In [658]:
[a*x for x in JJ]

[0, 0, -1/12*J2bar^4]

In [631]:
expr_to_monoms = lambda expr: eval(re.sub('x([0-9]+)([\\^]*)', lambda m: f'DD[{int(m[1])-1}]'+('**' if m[2]=='^' else ''), expr)).monomials()
[[expr_to_monoms(x) for x in y['OIDEAL']] for y in doc['OPLANES']]

[[[], []], [[], []]]

In [627]:
DD[1]*DD[2], DD[0]*DD[3]

(J0bar*J1bar, J0bar*J1bar)

In [345]:
A.sheaves.divisor(x0[0]*x0[5]).ch()

[1/4*z3^2*z5^2 + 1/2*z3^2*z5 + 1/2*z3*z5^2 + 1/2*z3^2 + z3*z5 + 1/2*z5^2 + z3 + z5 + 1]

In [644]:
R = PolynomialRing(QQ, [f'D{i}' for i in range(len(dresverts))] + [f'J{i}' for i in range(h11)])
D, J = R.gens()[:len(dresverts)], R.gens()[len(dresverts):]

In [645]:
lin_ideal = dresverts.T.dot(D).tolist()
sr_ideal = get_sr_ideal(D, dresverts, triang)
basis_ideal = list(J-to_mat(doc['DTOJ']).dot(D))
chow_ideal = lin_ideal + sr_ideal + basis_ideal

In [654]:
RR = R.quotient_ring(R.ideal(chow_ideal))
DD, JJ = RR.gens()[:len(dresverts)], RR.gens()[len(dresverts):]

In [592]:
doc['INVOL']

'{D1->D5,D5->D1,D2->D4,D4->D2,D3->D6,D6->D3}'

In [594]:
rescws

array([[0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1]])

In [593]:
D, DD

((D0, D1, D2, D3, D4, D5), (J1bar, J1bar, J0bar, J0bar, J0bar, J1bar))

In [289]:
term_patt = re.compile('([^+-]+)')
ind_patt = re.compile('x([0-9]+)')

term_to_deg = lambda term: sum(rescws[int(ind)-1] for ind in ind_patt.findall(term)).tolist()
term_to_monom = lambda term: sum(z[int(ind)-1] for ind in ind_patt.findall(term))

In [292]:
for oplane in doc['OPLANES']:
    odegs = []
    odivs = []
    for expr in oplane['OIDEAL']:
        term_degs = list(map(term_to_deg, term_patt.findall(expr)))
        term_divs = list(map(term_to_div, term_patt.findall(expr)))
        assert all(x == term_degs[0] for x in term_degs)
        assert all(x == term_divs[0] for x in term_divs)
        odegs.append(term_degs[0])
        odivs.append(term_divs[0])
    oplane['ODEGS'] = odegs
    oplane['ODIVS'] = odivs

In [548]:
doc['OPLANES']

[{'OIDEAL': ['x1*x4-x2*x5', 'x1*x3-x5*x6'], 'ODIM': 5}]

In [470]:
expr_to_monoms = lambda expr: eval(re.sub('x([0-9]+)([\\^]*)', lambda m: f'z{int(m[1])-1}'+('**' if m[2]=='^' else ''), expr)).monomials()

In [431]:
c_O = 1
O = 1
for x in doc['OPLANES'][0]['OIDEAL']:
    _c_O = 0
    _O = 0
    for y in expr_to_monoms(x):
        _c_O += A.sheaves.divisor(y).ch().part_of_degree(1)
        _O += A.sheaves.divisor(y).cohomology_class()
    c_O *= _c_O
    O *= _O
A.integrate(c_O*O)

118

In [433]:
a, b = expr_to_monoms(x)

In [441]:
q = A.sheaves.line_bundle(A.sheaves.divisor(a)).direct_sum(A.sheaves.line_bundle(A.sheaves.divisor(b)))

In [498]:
a = [expr_to_monoms(x)[0] for x in doc['SYMCYPOLY']]
b = [expr_to_monoms(x)[0].reduce(A.Stanley_Reisner_ideal()) for x in doc['SYMCYPOLY']]

In [499]:
sum(a).number_of_terms(), sum(b).number_of_terms()

(100, 65)

In [559]:
to_mat(re.sub('D([0-9]+)->D([0-9]+)', lambda x: '{'+f'{int(x[1])-1},{int(x[2])-1}'+'}', doc['INVOL']))

array([[0, 4],
       [4, 0],
       [1, 3],
       [3, 1],
       [2, 5],
       [5, 2]])

In [549]:
doc['OPLANES']

[{'OIDEAL': ['x1*x4-x2*x5', 'x1*x3-x5*x6'], 'ODIM': 5}]

In [565]:
m = Matrix([rescws[0]-rescws[4], rescws[1]-rescws[3]])#, rescws[2]-rescws[5]])

In [566]:
m.left_kernel()

Free module of degree 2 and rank 1 over Integer Ring
Echelon basis matrix:
[ 1 -1]

In [568]:
a = list(mdb.TRIANG.find({"POLYID": 288, "GEOMN": 1}))

In [573]:
b = list(mdb.INVOL.find({"POLYID": 288, "GEOMN": 1}))

In [577]:
[(x['TRIANGN'], x['INVOL'], x['VOLFORMPARITY'], x['ITENSXDINVOL'], x['SRINVOL']) for x in b], len(a)

([(1, '{D2->D4,D4->D2,D7->D8,D8->D7}', -1, True, True),
  (2, '{D2->D4,D4->D2,D7->D8,D8->D7}', -1, True, False),
  (3, '{D2->D4,D4->D2,D7->D8,D8->D7}', -1, True, False),
  (4, '{D2->D4,D4->D2,D7->D8,D8->D7}', -1, True, True),
  (5, '{D2->D4,D4->D2,D7->D8,D8->D7}', -1, True, True)],
 5)

In [580]:
mdb.INVOL.find_one({"POLYID":287, "GEOMN":1, "TRIANGN":1, "INVOLN":3})

{'_id': ObjectId('58c2781d27a728f655bf4da3'),
 'TRIANGN': 1,
 'POLYID': 287,
 'H11': 4,
 'OLDGEOMN': 1,
 'OLDTRIANGN': 1,
 'INVOL': '{D1->D2,D2->D1,D3->D4,D4->D3,D5->D6,D6->D5,D7->D8,D8->D7}',
 'INVOLN': 3,
 'GEOMN': 1,
 'divhodgeMARK': True,
 'divhodgeSTATS': {'MAXRSS': 1220000,
  'BSONSIZE': 64,
  'MAXVMSIZE': 186212000,
  'CPUTIME': 93},
 'INVOLDIVCOHOM': ['{1,0,0,2}', '{1,0,0,12}', '{1,0,0,12}', '{1,0,0,10}'],
 'CYPOLY': ['x1^2*x2^2*x3^3*x4^3',
  'x1*x2^3*x4^3*x5^3',
  'x1^2*x2^2*x3^2*x4^2*x5*x6',
  'x1^2*x2^2*x3*x4*x5^2*x6^2',
  'x1^3*x2*x3^3*x6^3',
  'x1^2*x2^2*x5^3*x6^3',
  'x1*x2^2*x3*x4^2*x5^2*x7',
  'x1^2*x2*x3^3*x4*x6*x7',
  'x1*x2^2*x4*x5^3*x6*x7',
  'x1^2*x2*x3^2*x5*x6^2*x7',
  'x1*x2^2*x3*x4^3*x5*x8',
  'x1*x2^2*x4^2*x5^2*x6*x8',
  'x1^2*x2*x3^2*x4*x6^2*x8',
  'x1^2*x2*x3*x5*x6^3*x8',
  'x1*x2*x3^2*x4*x5*x7^2',
  'x1*x2*x3*x5^2*x6*x7^2',
  'x1*x2*x3^2*x4^2*x7*x8',
  'x1*x2*x3*x4*x5*x6*x7*x8',
  'x1*x2*x5^2*x6^2*x7*x8',
  'x1*x2*x3*x4^2*x6*x8^2',
  'x1*x2*x4*x5*x6^2*x8^2',

In [581]:
len(list(mdb.INVOL.find({"H11":4, "orientifoldMARK":True})))

1777

In [583]:
len(list(mdb.INVOL.find({"H11":4})))

1778

In [584]:
1777/1778

0.999437570303712